# **Customized Toolkit**

### **When Do You Need a Customized Toolkit?**

If you have developed multiple related tools and want to group them together for easier management, creating a customized toolkit is the way to go. A customized toolkit is essentially a subclass of the `ToolkitBase` class with a predefined set of tools already added. This approach is useful when you know the tools required for a specific task or use case, and you want to make them readily available for your Tool Agents without manually adding each tool every time.

---

### **Steps to Create a Custom Toolkit**

1. **Define Your Custom Tools**: Develop the tools you want to include in the toolkit. These tools should inherit from the `ToolBase` class.  
2. **Subclass ToolkitBase**: Create a new class that inherits from `ToolkitBase`.
3. **Pre-add Tools**: Initialize your toolkit with the predefined tools in the constructor of the subclass.
4. **Additional Customization**: Add methods or attributes specific to the custom toolkit if needed.

---

### **Example: DataProcessingToolkit**

Imagine you are building a toolkit for data processing that includes:
- A tool for extracting key words.
- A tool for counting words.

Here’s how you can create a `DataProcessingToolkit`.

---
### **Step 1: Create Custom Tools**

**KeywordExtractorTool**

Here we will create a `KeywordExtractorTool` class that processes input `text` and identifies the most frequently occurring words as keywords. You can adjust the `max_keywords` parameter to limit the number of extracted keywords.

In [14]:
from swarmauri.tools.base.ToolBase import ToolBase
from swarmauri.tools.concrete.Parameter import Parameter
from typing import List, Literal, Dict
from pydantic import Field

class KeywordExtractorTool(ToolBase):
    version: str = "1.0.0"
    parameters: List[Parameter] = Field(
        default_factory=lambda: [
            Parameter(
                name="text",
                type="string",
                description="The text to extract keywords from.",
                required=True,
            ),
            Parameter(
                name="max_keywords",
                type="integer",
                description="The maximum number of keywords to extract.",
                required=False,
                default=5,
            ),
        ]
    )
    name: str = "KeywordExtractorTool"
    description: str = "Extracts key phrases from a given text."
    type: Literal["KeywordExtractorTool"] = "KeywordExtractorTool"

    def __call__(self, text: str, max_keywords: int = 5) -> Dict[str, List[str]]:
        try:
            words = text.split()
            word_frequency = {}

            for word in words:
                word = word.strip('.,!?()[]{}').lower()
                if word not in word_frequency:
                    word_frequency[word] = 0
                word_frequency[word] += 1

            # Sort by frequency, and take the top `max_keywords`
            sorted_words = sorted(word_frequency.items(), key=lambda item: item[1], reverse=True)
            keywords = [word for word, freq in sorted_words[:max_keywords]]

            return {"keywords": keywords}
        except Exception as e:
            return {"error": f"An error occurred: {str(e)}"}

**WordCounterTool**  

The `WordCounterTool` class Counts the number of words in a given text.

In [15]:
class WordCounterTool(ToolBase):
    version: str = "1.0.0"
    parameters: List[Parameter] = Field(
        default_factory=lambda: [
            Parameter(
                name="text",
                type="string",
                description="The text to count words in.",
                required=True,
            )
        ]
    )
    name: str = "WordCounterTool"
    description: str = "Counts the number of words in a given text."
    type: Literal["WordCounterTool"] = "WordCounterTool"

    def __call__(self, text: str) -> Dict[str, int]:
        word_count = len(text.split())
        return {"word_count": word_count}

---

### **Custom Toolkit Class**

we can now create the custom toolkit `DataProcessingToolkit` class which should be a sub-class of `ToolkitBase` class 

In [16]:
from swarmauri.toolkits.base.ToolkitBase import ToolkitBase

class DataProcessingToolkit(ToolkitBase):
    def __init__(self):
        super().__init__()
        # Add tools to the toolkit
        self.add_tool(KeywordExtractorTool())
        self.add_tool(WordCounterTool())


---
### **Using the DataProcessingToolkit**

In [18]:
# Initialize the custom toolkit
data_toolkit = DataProcessingToolkit()

# List tools in the toolkit
print(data_toolkit.tools.keys())

# Use the KeywordExtractorTool
keyword_extractor = data_toolkit.get_tool_by_name("KeywordExtractorTool")
text = "Artificial intelligence is revolutionizing industries. AI enables automation, enhances decision-making, and creates innovative solutions."
keyword_result = keyword_extractor(text=text, max_keywords=3)
print(keyword_result)


# Use the WordCounterTool
word_counter = data_toolkit.get_tool_by_name("WordCounterTool")
word_count_result = word_counter(text="This is a sample text to count words.")
print(word_count_result)


dict_keys(['KeywordExtractorTool', 'WordCounterTool'])
{'keywords': ['artificial', 'intelligence', 'is']}
{'word_count': 8}


---

### **Advantages of a Customized Toolkit**

1. **Pre-configured Setup**: Tools are already added, reducing setup time for repetitive tasks.
2. **Reusability**: Custom toolkits can be reused across different parts of your application or by multiple Swarm Agents.
3. **Consistency**: Ensures that related tools are always grouped together and readily accessible.

### **Extending Custom Toolkits**

You can further enhance your custom toolkit by:
- Adding specific utility methods.
- Overriding existing methods to implement custom behaviors.
- Incorporating validation to ensure compatibility of tools within the toolkit.

This approach simplifies tool management while offering the flexibility to adapt the toolkit to specific use cases.  
